In [29]:

import random
from dataclasses import dataclass


# Jocs

En aquesta pràctica programarem agents que puguen jugar a jocs. Recordeu que les característiques dels jocs en els que ens centrarem són:

* Jocs de suma zero.
* Dos jugadors alternats.
* Informació perfecta.
* Successors deterministes.
* Horitzó finit (per tant, no hi ha cicles).

## Joc del tres en ratlla

El tres en ratlla és un joc molt senzill, però que ens permetrà introduir els conceptes bàsics de la programació de jocs. El tabler és un tauler de 3x3 caselles. Cada jugador té una fitxa, que pot posar en una casella buida. El jugador que aconseguixi posar tres fitxes en línia (horitzontal, vertical o diagonal) guanya la partida. Si el taulell s'omple sense que cap jugador hagi aconseguit fer tres en ratlla, la partida acaba en empat.

### Representació dels estats

1. Per representar els estats del joc implementa una `Data Class` anomenada `EstatJoc`. Aquesta classe tindrà quatre atributs:

* `jugador`: el jugador que ha de fer el següent moviment.
* `utilitat`: mesura de la utilitat de l'estat pel jugador.
* `tauler`: una representació del tauler de joc.
* `moviments`: els moviments possibles des de l'estat actual.

In [30]:
@dataclass(frozen=True)
class EstatJoc:
    torn: str
    utilitat: int
    tauler: dict
    moviments: list

### Classe `TresEnRatlla`

El nostres jocs heretaran de la classe `Joc` que implementa els mètodes bàsics per a la programació de jocs. Aquesta classe té els següents mètodes:

* `accions(self, estat)`: retorna els moviments possibles des de l'estat passat com a paràmetre.
* `resultat(self, estat, accio)`: retorna l'estat que resulta d'aplicar l'accio a l'estat.
* `es_terminal(self, estat)`: retorna `True` si l'estat és terminal.
* `utilitat(self, estat, jugador)`: retorna la utilitat de l'estat per al jugador.

In [31]:
class Joc:
    """Un joc és similar a un problema, però té una utilitat per a cada estat i un test terminal en lloc d'un cost de ruta i un test de l'objectiu. Per crear un joc, subclasseu aquesta classe i implementeu accions, resultats, utilitat, i test_terminal. Podeu sobreescriure la visualització i els successors o bé podeu heretar els seus mètodes per defecte. També necessitareu establir l'atribut .inicial a l'estat inicial; això es pot fer en el constructor."""

    def accions(self, estat):
        """Retorna una llista dels moviments permesos en aquest punt."""
        raise NotImplementedError

    def resultat(self, estat, moviment):
        """Retorna l'estat que resulta de fer un moviment des d'un estat."""
        raise NotImplementedError

    def utilitat(self, estat, jugador):
        """Retorna el valor d'aquest estat final per al jugador."""
        raise NotImplementedError

    def test_terminal(self, estat):
        """Retorna True si aquest és un estat final per al joc."""
        return not self.accions(estat)

    def torn(self, estat):
        """Retorna el jugador al qual li toca moure en aquest estat."""
        return estat.torn

    def display(self, estat):
        """Imprimeix o mostra d'alguna manera l'estat."""
        print(estat)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def juga_joc(self, *jugadors):
        """Juga un joc de n-persones, alternant torns."""
        estat = self.inicial
        while True:
            for jugador in jugadors:
                moviment = jugador(self, estat)
                estat = self.resultat(estat, moviment)
                self.display(estat)
                if self.test_terminal(estat):
                    self.display(estat)
                    print()
                    return self.utilitat(estat, self.torn(self.inicial))

2. Has de crear una classe anomenada `TresEnRatlla` que herete de la classe `Joc`. Has de tindre en compte les següents consideracions:

* Els jugadors es representaran amb les cadenes de caràcters `'X'` i `'O'`.
* Els moviments es representaran amb tuples de dos elements, que seran les coordenades de la casella on es posa la fitxa.
* El tauler es representarà amb un diccionari que té com a clau la tupla de coordenades i com a valor el jugador que té la fitxa en eixa casella.
* La utilitat serà 1 si guanya el jugador `'X'`, -1 si guanya el jugador `'O'` i 0 en cas d'empat.
* L'estat inicial tindrà el jugador `'X'`, el tauler estarà buit i els moviments seran totes les caselles del tauler.
* Per facilitar la implementació de jocs més grans, el constructor de la classe tindrà tres paràmetres opcionals: `h`, `v` i `k`. `h` i `v` són les dimensions del tauler i `k` és el nombre de fitxes en ratlla per a guanyar. Per defecte, `h=3`, `v=3` i `k=3`.

In [32]:
class TresEnRatlla(Joc):
    """Joc del tres en ratlla (generalitzat a k) en un tauler h*v on guanya el primer que aconsegueix k en ratlla.
    Un estat es representa com una tupla (torn, tauler, moviments) on:
        - torn és el jugador a moure ('X' o 'O')
        - utilitat és la utilitat d'eixe estat, precalculada per eficiència.
        - moviments és la llista de caselles buides del tauler
        - tauler és un diccionari que conté per cada casella (x, y) del tauler el jugador que té la fitxa en eixa casella
        - moviments és la llista de caselles buides del tauler
    """

    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        moviments = [(x, y) for x in range(1, h + 1)
                     for y in range(1, v + 1)]
        self.inicial = EstatJoc(
            torn='X', utilitat=0, tauler={}, moviments=moviments
        )

    def accions(self, estat):
        """Les accions legals en un estat són les caselles buides."""
        return estat.moviments

    def resultat(self, estat, moviment):
        """Retorna l'estat que resulta d'aplicar un moviment a un estat."""
        if moviment not in estat.moviments:
            return estat  # El moviment il·legal no té cap efecte
        tauler = estat.tauler.copy()
        tauler[moviment] = estat.torn
        moviments = list(estat.moviments)
        moviments.remove(moviment)
        return EstatJoc(
            torn=('O' if estat.torn == 'X' else 'X'),
            utilitat=self.calcula_utilitat(
                tauler, moviment, estat.torn
            ),
            tauler=tauler, moviments=moviments
        )

    def utilitat(self, estat, jugador):
        """Retorna la utilitat d'un estat per a un jugador. El valor és 1 si el jugador guanya; -1 si el contrari guanya; 0 en cas d'empat."""

        return estat.utilitat if jugador == 'X' else -estat.utilitat

    def test_terminal(self, estat):
        """Un estat és terminal si algun jugador ha guanyat o no queden moviments."""
        return estat.utilitat != 0 or len(estat.moviments) == 0

    def display(self, estat):
        """Mostra l'estat actual del joc."""
        tauler = estat.tauler
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(tauler.get((x, y), '.'), end=' ')
            print()

    def calcula_utilitat(self, tauler, estat, jugador):
        """Si 'X' guanya amb aquest moviment, retorna 1; si 'O' guanya retorna -1; si no retorna 0."""
        if (self.k_en_ralla(tauler, estat, jugador, (0, 1)) or
                self.k_en_ralla(tauler, estat, jugador, (1, 0)) or
                self.k_en_ralla(tauler, estat, jugador, (1, -1)) or
                self.k_en_ralla(tauler, estat, jugador, (1, 1))):
            return +1 if jugador == 'X' else -1
        else:
            return 0

    def k_en_ralla(self, tauler, moviment, jugador, delta_x_y, new_k=None):
        """Retorna veritat si hi ha una línia a través del moviment al tauler per al jugador.
        
        """
        (delta_x, delta_y) = delta_x_y
        x, y = moviment
        n = 0  # n és el nombre de moviments en línia
        while tauler.get((x, y)) == jugador:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = moviment
        while tauler.get((x, y)) == jugador:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Perquè hem comptat dos cops el mateix moviment
        return n >= self.k

### Funció `jugador_aleatori`

Per a poder jugar al tres en ratlla, necessitem un jugador aleatori. Aquest jugador tria un moviment aleatori entre els moviments possibles.

3. Implementa la funció `jugador_aleatori` que rep com a paràmetres un joc i un estat i retorna un moviment aleatori entre els moviments possibles.


In [33]:
def jugador_aleatori(joc, estat):
    return random.choice(joc.accions(estat))

4. Comprova que el jugador aleatori funciona correctament jugant una partida contra ell mateix. Per a això, utilitza la funció `juga_joc` de la classe `Joc`.

In [34]:
joc = TresEnRatlla()
joc.juga_joc(jugador_aleatori, jugador_aleatori)

. . . 
. . X 
. . . 
. O . 
. . X 
. . . 
X O . 
. . X 
. . . 
X O . 
. . X 
O . . 
X O . 
. X X 
O . . 
X O O 
. X X 
O . . 
X O O 
X X X 
O . . 
X O O 
X X X 
O . . 


1

### Funció `minimax`

5. Implementa la funció `minimax` que rep com a paràmetres un joc i un estat i retorna un moviment que té en compte la utilitat dels estats resultants de fer els moviments possibles. 

In [35]:
def minimax(joc, estat):
    def maximitza(joc, estat):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = -float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = minimitza(joc, joc.resultat(estat, accio))
            if v2 > v:
                v = v2
                moviment = accio
        return v, moviment

    def minimitza(joc, estat):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = maximitza(joc, joc.resultat(estat, accio))
            if v2 < v:
                v = v2
                moviment = accio
        return v, moviment

    return maximitza(joc, estat)[1]

6. Comprova que el jugador `minimax` funciona correctament jugant una partida contra el jugador aleatori. `minimax` sempre hauria de guanyar o empatar. 

In [36]:
joc = TresEnRatlla()
joc.juga_joc(minimax, jugador_aleatori)

X . . 
. . . 
. . . 
X . . 
. . O 
. . . 
X . X 
. . O 
. . . 
X . X 
. . O 
. O . 
X X X 
. . O 
. O . 
X X X 
. . O 
. O . 


1

### Funció `minimax_alphabeta`

7. El cost computacional de l'algorisme `minimax` és molt alt. Per això, implementa la funció `minimax_alphabeta` que utilitza l'algorisme `minimax` amb l'optimització alfa-beta.
    Testeja la funció `minimax_alphabeta` jugant una partida contra el jugador aleatori. `minimax_alphabeta` sempre hauria de guanyar o empatar.

In [37]:
def minimax_alphabeta(joc, estat):
    def maximitza(joc, estat, alfa, beta):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = -float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = minimitza(joc, joc.resultat(estat, accio), alfa, beta)
            if v2 > v:
                v = v2
                moviment = accio
            if v >= beta:
                return v, moviment
            alfa = max(alfa, v)
        return v, moviment

    def minimitza(joc, estat, alfa, beta):
        if joc.test_terminal(estat):
            return joc.utilitat(estat, joc.torn(joc.inicial)), None
        v = float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = maximitza(joc, joc.resultat(estat, accio), alfa, beta)
            if v2 < v:
                v = v2
                moviment = accio
            if v <= alfa:
                return v, moviment
            beta = min(beta, v)
        return v, moviment

    return maximitza(joc, estat, -float('inf'), float('inf'))[1]


joc = TresEnRatlla()
joc.juga_joc(minimax_alphabeta, jugador_aleatori)

X . . 
. . . 
. . . 
X . O 
. . . 
. . . 
X . O 
X . . 
. . . 
X . O 
X . . 
. O . 
X . O 
X X . 
. O . 
X . O 
X X O 
. O . 
X . O 
X X O 
X O . 
X . O 
X X O 
X O . 


1

## Quatre en ratlla

El quatre en ratlla és un joc molt similar al tres en ratlla, però en un tauler de 7x6 caselles i amb 4 fitxes en ratlla per a guanyar. Les opcions de moviment són les columnes del tauler. Quan es tria una columna, la fitxa cau fins a la posició més baixa que estiga buida.

8. Modifica la classe `TresEnRatlla` per a implementar el joc del quatre en ratlla. Testeja el funcionament utilitza la funció `juga_joc` de la classe `Joc` i dos jugador aleatoris.

In [38]:
class QuatreEnRatlla(TresEnRatlla):
    """Un joc de tipus Tres en ratlla en el qual només pots fer un moviment a la fila inferior,
    o en una casella directament per sobre d'una casella ocupada.  Tradicionalment
    es juga en un tauler de 7x6 i es requereixen 4 en línia."""

    def __init__(self, h=7, v=6, k=4):
        TresEnRatlla.__init__(self, h, v, k)

    def accions(self, state):
        return [(x, y) for (x, y) in state.moviments
                if y == 1 or (x, y - 1) in state.tauler]

    def display(self, estat):
        """Mostra l'estat actual del joc."""
        tauler = estat.tauler
        for y in range(self.v, 0, -1):
            for x in range(1, self.h + 1):
                print(tauler.get((x, y), '.'), end=' ')
            print()

In [39]:
joc = QuatreEnRatlla()
joc.juga_joc(jugador_aleatori, jugador_aleatori)

. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . O . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . O . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . O O . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . O O . X . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . O O O X . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . X . 
X . O O O X . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . O . 
X . . . . X . 
X . O O O X . 
. . . . . . . 
. . . . . . . 
. . . . . X . 
. . . . . O . 
X . . . . X . 
X . O O O X . 
. . . . . . . 
. . . . . . . 
. . . . . X . 
. . . . . O . 
X . . . . X . 
X . O O O X O 
. . . . . . . 
. . . . . . . 
. . . . . X . 
. . . . . O . 
X . . X . X . 
X . O O O X O 
. . . . . 

-1

### Funció minimax amb profunditat limitada i avaluació heurística

9. Implementa la funció `minimax_profunditat_limitada` que rep com a paràmetres un joc, un estat i una profunditat i retorna un moviment que té en compte la utilitat dels estats resultants de fer els moviments possibles fins a la profunditat indicada. Aquesta funció ha de tindre en compte que si l'estat és terminal o s'ha arribat a la profunditat límit, s'ha d'avaluar l'estat utilitzant la funció `avaluacio`.
    Implementa també la funció `avaluacio` que rep com a paràmetres un joc i un estat i retorna un valor numèric que indica la qualitat de l'estat per al jugador que té el torn. Pots utilitzar la funció `avaluacio` que es mostra a continuació o implementar una altra funció diferent.


In [40]:
def avaluacio(joc, estat):
    """Retorna un valor numèric que indica la qualitat de l'estat per al jugador que té el torn."""
    def num_k_en_ralla(tauler, jugador, delta_x_y, k):
        """Retorna el nombre de línies de k fitxes en ratlla del jugador en el tauler.
        """
        (delta_x, delta_y) = delta_x_y
        n = 0  # n és el nombre de moviments en línia
        for x in range(1, joc.h + 1):
            for y in range(1, joc.v + 1):
                if tauler.get((x, y)) == jugador:
                    # Comptar les fitxes en línia des d'ací
                    n += 1
                    for m in range(1, k):
                        if tauler.get((x + m * delta_x, y + m * delta_y)) != jugador:
                            n -= 1
                            break
                    if n == k:
                        return 1
                n = 0
        return 0
    
    num = 0
    for i in range(1, joc.k + 1):
        aux = 0
        aux += num_k_en_ralla(estat.tauler, estat.torn, (0, 1), i)
        aux += num_k_en_ralla(estat.tauler, estat.torn, (1, 0), i)
        aux += num_k_en_ralla(estat.tauler, estat.torn, (1, -1), i)
        aux += num_k_en_ralla(estat.tauler, estat.torn, (1, 1), i)
        num += aux * 10 ** i
    return num

In [41]:
def minimax_ab_profunditat_limitada(joc, estat, profunditat=6):
    def maximitza(joc, estat, alfa, beta, profunditat):
        if joc.test_terminal(estat) or profunditat == 0:
            return avaluacio(joc, estat), None
        v = -float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = minimitza(joc, joc.resultat(estat, accio), alfa, beta, profunditat - 1)
            if v2 > v:
                v = v2
                moviment = accio
            if v >= beta:
                return v, moviment
            alfa = max(alfa, v)
        return v, moviment

    def minimitza(joc, estat, alfa, beta, profunditat):
        if joc.test_terminal(estat) or profunditat == 0:
            return avaluacio(joc, estat), None
        v = float('inf')
        moviment = None
        for accio in joc.accions(estat):
            v2, _ = maximitza(joc, joc.resultat(estat, accio), alfa, beta, profunditat - 1)
            if v2 < v:
                v = v2
                moviment = accio
            if v <= alfa:
                return v, moviment
            beta = min(beta, v)
        return v, moviment

    return maximitza(joc, estat, -float('inf'), float('inf'), profunditat)[1]

In [42]:
joc = QuatreEnRatlla()
joc.juga_joc(minimax_ab_profunditat_limitada, jugador_aleatori)

. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
O . . . . . . 
X . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
O . . . . . . 
X . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
O . . . . . . 
X . O . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . . . . 
O . . . . . . 
X . O . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . . . . 
O . . . . . . 
X . O . O . . 
. . . . . . . 
X . . . . . . 
X . . . . . . 
X . . . . . . 
O . . . . . . 
X . O . O . . 
. . . . . . . 
X . . . . . . 
X . . . . . . 
X . . . . . . 
O . . . . . . 
X . O O O . . 
X . . . . . . 
X . . . . . . 
X . . . . . . 
X . . . . . . 
O . . . . . . 
X . O O O . . 
X . . . . . . 
X . . . . . . 
X . . . . . . 
X . . . . . . 
O . . . . . . 
X . O O O . . 


1

### Jugador humà

10. Implementa la funció `jugador_huma` que rep com a paràmetres un joc i un estat i retorna un moviment triat per l'usuari. Si el moviment triat no és vàlid, s'ha de demanar un altre moviment fins que es triï un moviment vàlid.
    Intenta jugar una partida contra el jugador que hem anat creant.

In [43]:
def jugador_huma(joc, estat):
    moviment = None
    while moviment not in joc.accions(estat):
        columna = eval(input('Columna? '))
        altura = max([y for (x, y) in estat.tauler if x == columna] + [0])
        moviment = (columna, altura + 1)
    return moviment

In [47]:
joc = QuatreEnRatlla()
joc.juga_joc(minimax_ab_profunditat_limitada, jugador_aleatori)

. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . O . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . . O . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . O O . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . . . . 
X . . . O O . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . O . . 
X . . . O O . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . . . . 
X . . . O . . 
X . . . O O . 
. . . . . . . 
. . . . . . . 
X . . . . . . 
X . . . . . . 
X . . . O . . 
X . . . O O . 


1